In [13]:
'''1. Remove foreign records to a 'foreign dataframe' and delete them from this df. - applicable for  address
2.Find whether all phone numbers are empty, if empty drop those rows (8708)
3.Column Phone matches the Phone number in preferred phone number
Transfer records with non-null values in Phone to a new dataframe with columns EID and phone
Find the count of records (124000)
4. Null values in Phone: (36000 records) Create a new df:Check the content in preferred phone 
and grasp the preferred phone and add it to df with columns eid and phone

'''

import sys
import pandas as pd
from fuzzywuzzy import fuzz

df_txexAddr = pd.read_csv('TexasExes.csv')
print("original count ",len(df_txexAddr))
df_txexAddr = df_txexAddr[['EID', 'Mailing Street', 'Mailing City', 'Mailing State/Province', 'Mailing Zip/Postal Code', 'Mailing Country', 'Primary Address Type', 'TE_Current_Address_Updated', 'Other Street', 'Other City', 'Other State/Province', 'Other Zip/Postal Code', 'Other Country']]

df_Addr = df_txexAddr.dropna(how='all', subset=['Mailing Street'])
print("Count records after dropping records with no address : ",len(df_Addr))
#print(df_Addr.head(3))

df_Addr = df_Addr[['EID', 'Mailing Street']]
df_Addr['EID'] = df_Addr['EID'].str.upper()
print(len(df_Addr))
print(df_Addr.head(3))

original count  2200
Count records after dropping records with no address :  1956
1956
        EID                  Mailing Street
0    TO3269  5130 Fulton Street Apartment 3
1  0563NUMT              8406 La Plata Loop
2     KSN99                      PO Box 517


In [14]:
# dividing the large dataframe into multiple small dataframes
df_addr_csv = df_Addr
print(len(df_addr_csv))
print(df_addr_csv.head(3))
print(df_addr_csv.tail(3))

1956
        EID                  Mailing Street
0    TO3269  5130 Fulton Street Apartment 3
1  0563NUMT              8406 La Plata Loop
2     KSN99                      PO Box 517
           EID           Mailing Street
2196  0733MAUD        516 Saginaw Court
2197  VIELMAVM  5728 Cedar Grove Circle
2199   BLUMJR1        30520 Lynne Drive


In [16]:
raw_eid = df_addr_csv['EID'].dropna()
eid = raw_eid.tolist()
#print(len(eid))
# How many elements each list should have 
n = 1000
pid = {}
for i in range((len(eid) + n - 1) // n ):
    #globals()["eid" + str(i)] = eid[i * n:(i + 1) * n]
    pid[i] = eid[i * n:(i + 1) * n]

In [18]:
import cx_Oracle
connection = cx_Oracle.connect("xxxx/Zyyy@ea.y.z.com:1525/z.x.y.com")
cursor = connection.cursor()
df_addr_db = pd.DataFrame(columns = ['EID','Address1','Address2','City','State','Zip Code'])
for i in range(2):
    cursor.execute("select UT_EID, vi_addr_line1, vi_addr_line2, vi_city, vi_state,vi_zip from vi_vips_main m, vi_vips_addresses a  where m.ALUMNI_ID = a.ALUMNI_ID AND m.ut_eid IN %s" % str(tuple(pid[i])).replace(',)',')'))
    results = cursor.fetchall()
    df1 = pd.DataFrame(results, columns = ['EID','Address1','Address2','City','State','Zip Code'])
    df_addr_db = df_addr_db.append(df1,ignore_index=True)
cursor.close()
connection.close()                   

In [19]:
print(len(df_addr_db))
df_addr_db = df_addr_db.drop_duplicates()
print(len(df_addr_db))
print(df_addr_db.head(3))

2276
2275
        EID              Address1 Address2        City State     Zip Code
0  0222BVZW  4928 Brookburn Drive     None   San Diego    CA  92130277828
1  0229PXTR         P. O. Box 965     None  Gatesville    TX  76528096565
2  0229VVHV          2515 Redondo     None    Amarillo    TX  79107641115


In [20]:
addr_diff = pd.DataFrame()
for i in range(len(df_addr_csv)):
#for i in (11,44,55,66,123,133,143,333,445,667,246,988):
    flag = 0
    id_csv = df_addr_csv.iloc[i,0]
    csv_addr = df_addr_csv.iloc[i,1]
    cmp_dbaddr = df_addr_db.loc[df_addr_db['EID'] == id_csv]
    #print(cmp_dbaddr)
    #print(df_addr_csv.loc[df_addr_csv['EID'] == id_csv])
    #print("---")
    
    for j in range(len(cmp_dbaddr)):
        db_addr = cmp_dbaddr.iloc[j,1]
        Token_Set_Ratio = fuzz.token_set_ratio(csv_addr,db_addr)
        if (Token_Set_Ratio > 77):
            flag = 0
            break
        else:
            flag = 1
    
    #print("flag = ", flag)
    
    if (flag == 1):
        addrlist = []
        for k in range(len(cmp_dbaddr)):
            temp = (cmp_dbaddr.loc[cmp_dbaddr['EID'] == id_csv]).iloc[:,1]
            addrlist = temp.values.tolist()
        addr_diff = addr_diff.append({'EID':id_csv, 'DB_Address':addrlist,'CSV_Address':csv_addr}, ignore_index=True)
#print("----------------------------------------------------------------")
#print(addr_diff)
#print("----------------------------------------------------------------")

In [21]:
print(addr_diff.head(11))
print(len(addr_diff))

                              CSV_Address  \
0          5130 Fulton Street Apartment 3   
1                       201 N 36th Avenue   
2                       3470 Brentwood Dr   
3                     1712 Cherokee Place   
4                        6151 Ella Lee Ln   
5   8215 Cypresswood Drive Apartment 2104   
6                        148 Stanley Road   
7           2900 W Anderson Ln  Ste C-200   
8                        1510 College Ave   
9              3215 Exposition Blvd #B-21   
10                   13051 Whittington Dr   

                                           DB_Address       EID  
0   [4531 Nassau Drive, 3202 W Las Positas Boulevard]    TO3269  
1                                [1203 Market Street]  MORGANJP  
2                        [165 Central Caldwood Drive]   GGIGLIO  
3                                 [6635 Genstar Lane]    JWD654  
4                              [23540 Macedonia Road]     JJF96  
5              [Yonhuidong 55, 10710 Idlebrook Drive]     FLYML

In [22]:
addr_diff.to_csv(r'C:\Users\le5752\Documents\DB_Python\addr3.csv', index = False)